In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime

plt.rcParams.update({'font.size': 15})

In [2]:
#First get information about the database
path_and_file = 'data\\dataset_wildfires\\FPA_FOD_20170508.sqlite'

def get_table_names(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_list = cursor.fetchall()
    return [item[0] for item in table_list]

def get_columns(connection, table_name):
    cursor = connection.execute('select * from ' + table_name)
    return(list(map(lambda x: x[0], cursor.description)))

db = sqlite3.connect(path_and_file)
table_names = get_table_names(db)
tables = dict.fromkeys(table_names,[])
for table_name in table_names:
    print(table_name)
    try:
        columns = get_columns(db, table_name)
        tables[table_name] = columns
        print('    ',columns)
    except:
        print('    #no columns#')

spatial_ref_sys
     ['srid', 'auth_name', 'auth_srid', 'ref_sys_name', 'proj4text', 'srtext']
spatialite_history
     ['event_id', 'table_name', 'geometry_column', 'event', 'timestamp', 'ver_sqlite', 'ver_splite']
sqlite_sequence
     ['name', 'seq']
geometry_columns
     ['f_table_name', 'f_geometry_column', 'geometry_type', 'coord_dimension', 'srid', 'spatial_index_enabled']
spatial_ref_sys_aux
     ['srid', 'is_geographic', 'has_flipped_axes', 'spheroid', 'prime_meridian', 'datum', 'projection', 'unit', 'axis_1_name', 'axis_1_orientation', 'axis_2_name', 'axis_2_orientation']
views_geometry_columns
     ['view_name', 'view_geometry', 'view_rowid', 'f_table_name', 'f_geometry_column', 'read_only']
virts_geometry_columns
     ['virt_name', 'virt_geometry', 'geometry_type', 'coord_dimension', 'srid']
geometry_columns_statistics
     ['f_table_name', 'f_geometry_column', 'last_verified', 'row_count', 'extent_min_x', 'extent_min_y', 'extent_max_x', 'extent_max_y']
views_geometry_columns

In [3]:
# See how state info is presented
pd.read_sql_query('SELECT STATE, COUNT(OBJECTID) FROM Fires GROUP BY STATE', db)

,STATE,COUNT(OBJECTID)
0,AK,12843
1,AL,66570
2,AR,31663
3,AZ,71586
4,CA,189550
5,CO,34157
6,CT,4976
7,DC,66
8,DE,171
9,FL,90261


In [4]:
query = """
SELECT NWCG_REPORTING_AGENCY, COUNT(OBJECTID) AS 'Count'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY NWCG_REPORTING_AGENCY
ORDER BY Count"""

pd.read_sql_query(query, db)

,NWCG_REPORTING_AGENCY,Count
0,BOR,13
1,DOD,23
2,NPS,3819
3,FWS,4027
4,BIA,6959
5,BLM,20029
6,FS,43002
7,ST/C&L,111678


In [5]:
#NWCG_UnitIDActive_20170109: ['OBJECTID', 'UnitId', 'GeographicArea', 'Gacc', 'WildlandRole', 'UnitType', 'Department', 'Agency', 'Parent', 'Country', 'State', 'Code', 'Name']
query = """
SELECT *
FROM NWCG_UnitIDActive_20170109 
WHERE State = 'CA'
ORDER BY Agency
"""

pd.read_sql_query(query, db)

,OBJECTID,UnitId,GeographicArea,Gacc,WildlandRole,UnitType,Department,Agency,Parent,Country,State,Code,Name
0,584,USCAAEU,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,AEU,Amador-El Dorado Unit
1,588,USCAAGL,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,AGL,Angels Camp Fire Department
2,590,USCAAIR,CA,USCAONCC,Incident Host Geographic,Tribe,None,None,None,US,CA,AIR,Alturas Indian Rancheria Fire Service
3,629,USCABCR,CA,USCAOSCC,Incident Host Geographic,US State,None,None,None,US,CA,BCR,Big Creek Volunteer Fire Department
4,682,USCABTCC,CA,USCAONCC,Dispatch/Coordination Center,US State,None,None,None,US,CA,BTCC,Butte Command Center
5,685,USCABTU,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,BTU,Butte Unit
6,696,USCACAS,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,CAS,Governor's Office of the State of California
7,708,USCACCC,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,CCC,California Conservation Corps
8,719,USCACDC,CA,USCAONCC,Incident Host Geographic,US State,None,None,None,US,CA,CDC,Headquarters
9,721,USCACDF,CA,USCAONCC,Dispatch/Coordination Center,US State,None,None,None,US,CA,CDF,Headquarters


In [6]:
query = """
SELECT NWCG_REPORTING_AGENCY, NWCG_REPORTING_UNIT_NAME, COUNT(OBJECTID) AS 'Count'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY NWCG_REPORTING_UNIT_NAME
ORDER BY NWCG_REPORTING_AGENCY
"""

pd.read_sql_query(query, db)

,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_NAME,Count
0,BIA,Cabazon Tribe,12
1,BIA,Central California Agency,237
2,BIA,Colorado River Agency,24
3,BIA,Fort Yuma Agency,490
4,BIA,Hoopa Valley Tribe,4004
5,BIA,Northern California Agency,215
6,BIA,Pacific Regional Office,2
7,BIA,Round Valley Tribe,408
8,BIA,Southern California Agency,798
9,BIA,Tule River Agency,769


In [7]:
query = """
SELECT NWCG_REPORTING_AGENCY, COUNTY, COUNT(OBJECTID) AS 'Count'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY NWCG_REPORTING_AGENCY, COUNTY
ORDER BY NWCG_REPORTING_AGENCY, Count DESC"""

pd.read_sql_query(query, db)

,NWCG_REPORTING_AGENCY,COUNTY,Count
0,BIA,None,6958
1,BIA,Riverside County,1
2,BLM,None,3534
3,BLM,Riverside,3218
4,BLM,Kern,1235
5,BLM,Lassen,1051
6,BLM,San Diego,1035
7,BLM,El Dorado,859
8,BLM,Shasta,847
9,BLM,San Bernardino,844


In [8]:
query = """
SELECT upper(COUNTY) AS CountyName, count(OBJECTID) AS Count, sum(FIRE_SIZE) AS 'Area'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY CountyName
ORDER BY CountyName ASC"""

pd.read_sql_query(query, db)

,CountyName,Count,Area
0,None,133329,6.221475e+06
1,003,101,1.898497e+04
2,005,28,3.415000e+01
3,007,25,1.104610e+03
4,009,30,1.160270e+03
5,011,13,2.986640e+04
6,015,46,3.885975e+04
7,017,317,9.855231e+04
8,019,231,1.755690e+05
9,021,14,2.597000e+02


In [9]:
query = """
SELECT FIPS_CODE AS FIPS, max(upper(COUNTY)) AS CountyName, count(OBJECTID) AS Count, sum(FIRE_SIZE) AS 'Area'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY FIPS
ORDER BY CountyName ASC"""

pd.read_sql_query(query, db)

,FIPS,CountyName,Count,Area
0,None,None,133329,6.221475e+06
1,001,ALAMEDA,301,3.655510e+03
2,003,ALPINE,337,2.074393e+04
3,007,BUTTE,1768,4.668653e+04
4,009,CALAVERAS,1085,5.986990e+03
5,011,COLUSA,111,5.299725e+04
6,013,CONTRA COSTA,494,1.767740e+03
7,015,DEL NORTE,238,5.715425e+04
8,005,DOUGLAS,592,1.885742e+04
9,017,EL DORADO,2676,1.149268e+05


In [10]:
query = """
SELECT FIPS_CODE AS FIPS, max(upper(COUNTY)) AS CountyName, count(OBJECTID) AS Count, sum(FIRE_SIZE) AS 'Area'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY FIPS
ORDER BY Area DESC
"""

pd.read_sql_query(query, db)

,FIPS,CountyName,Count,Area
0,None,None,133329,6.221475e+06
1,105,TRINITY COUNTY,1255,5.829380e+05
2,093,SISKIYOU,2156,5.286323e+05
3,073,SAN DIEGO COUNTY,2443,5.215896e+05
4,035,LASSEN,1552,4.859005e+05
5,037,LOS ANGELES,2703,4.029847e+05
6,083,SANTA BARBARA COUNTY,236,3.848425e+05
7,053,MONTEREY COUNTY,660,2.884774e+05
8,111,VENTURA COUNTY,132,2.790186e+05
9,109,TUOLUMNE,1508,2.769512e+05


In [16]:
query = """
SELECT fire_year AS 'Year', count(objectid) AS 'Count', sum(fire_size)/1000000 AS 'Area (x10^6)'
FROM Fires 
WHERE STATE = 'CA'
GROUP BY Year
ORDER BY Year Asc
"""

pd.read_sql_query(query, db)

,Year,Count,Area (x10^6)
0,1992,10833,0.296440
1,1993,8270,0.321495
2,1994,8654,0.407945
3,1995,7381,0.216069
4,1996,9173,0.707110
5,1997,7930,0.324566
6,1998,6864,0.160013
7,1999,8910,0.804123
8,2000,6977,0.252676
9,2001,8182,0.350177


In [15]:
query = """
SELECT fire_year AS 'Year', count(objectid) AS 'Count', sum(fire_size)/1000000 AS 'Area in millions'
FROM Fires
GROUP BY Year
ORDER BY Year Asc
"""

pd.read_sql_query(query, db)

,Year,Count,Area in millions
0,1992,67975,2.199958
1,1993,61989,2.191696
2,1994,75955,4.116676
3,1995,71472,2.049585
4,1996,75574,6.005156
5,1997,61450,3.214635
6,1998,68370,2.011102
7,1999,89363,6.080200
8,2000,96416,7.639499
9,2001,86587,3.724072
